# COMP5046 Assignment 1
*Make sure you change the file name with your unikey.*

# Readme
*If there is something to be noted for the user, please mention here.* 

*If you are planning to implement a program with Object Oriented Programming style*

***Visualising the comparison of different results is a good way to justify your decision.***

# 1 - Data Preprocessing (Personality chat datasets)

## 1.1. Download Dataset (Personality chat datasets)

In [0]:
# Please comment your code
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#files - {google drive id : file name}
files = {"1DyV6vxpd5m2VjcWnBs1dR7txf7f3assJ":"qna_chitchat_the_comic.tsv",\
          "1jo8eb--b7Tj6nJpJ39eYCp4ZiL3Oz6sC":"qna_chitchat_the_friend.tsv",\
          "12x4QlCZ7nT9CJvOqioN-ZWTwvlKCwlkv":"qna_chitchat_the_professional.tsv"}

for fid,fname in files.items():
    downloaded = drive.CreateFile({'id':fid}) 
    downloaded.GetContentFile(fname)  

## 1.2. Preprocess data (Personality chat datasets)

Processing data is divided by several parts.
1. get data from 3 files and divided them to 4 lists that one is a list of question sentences (only one is because the questions of 3 files is same) and 3 lists of answer.

2. have written a method named "sentence_process" to process sentences including removing punctuations, tokenization, lowercase,  stemming and replacement of some special words (for example    's ---> is, 're -->are).






In [0]:
import pandas as pd
import numpy as np




filesname = list(files.values())
# read data by pandas
comic = pd.read_csv(filesname[0], sep='\t')
friend = pd.read_csv(filesname[1], sep='\t')
professional = pd.read_csv(filesname[2], sep='\t')


#Questions lists
Q = list(comic["Question"])
# friend_Q = list(friend["Question"])
# professional_Q = list(professional["Question"])

#answer lists
comic_A = list(comic["Answer"])
friend_A = list(friend["Answer"])
professional_A = list(professional["Answer"])


In [0]:
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords as sw
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [0]:
# removing punctuations, tokenization, lowercase, stemming
def remove_punctuation_re(x):
    x = re.sub(r'[^\w\s]','',x)  
    return x
  
lemmatizer = WordNetLemmatizer()

def sentence_process(S):
    replace_dic = {"'s" : "is","'re":"are","'m":"am","n't":"not","'ve":"have"}
    sentences_list = []
    for sentence in S:
        sentence_token = []

        sentences = word_tokenize(sentence.lower())
        for i in range(len(sentences)):
            word = sentences[i]
            if word in replace_dic.keys():
                word = replace_dic[word]
            word = remove_punctuation_re(word).strip()
            word = lemmatizer.lemmatize(word)
    
            sentence_token.append(word)
        sentence_token = [i for i in sentence_token if i!= '']
        sentences_list.append(sentence_token)
    return sentences_list


# 2 - Model Implementation

## 2.1. Word Embeddings




This program have implemented Word2Vec with Neural Network with TensorFlow to do word embddings. Because this method transfer the word to vector with strong relationship with the context of this word. 




### 2.1.1. Download Dataset for Word Embeddings



Part 2.1.1 is empty because this program use Microsoft BotBuilder Personality Chat Datasets that have downloaded as corpus of Word Embeddings.



In [0]:
'''
Part 2.1.1 is empty because this program use Microsoft BotBuilder Personality Chat Datasets that have downloaded as corpus of Word Embeddings.
'''

### 2.1.2. Data Preprocessing for Word Embeddings

1.  preprocess the sentence (tokenization,stemming ...) using sentence_process method. First we must do tokenization for sentence to word so that we can get word vector with relationship with nearlist word, which have better effects in training. Stemming is to can make words more concentrative because it can put the diffenernet conversion of same word together. Replacement of abbreviation(such as 's -->is')have same effects.



2. generate a dict for {uniqueword :index} -- necessary part




In [0]:
import tensorflow as tf
from functools import reduce
import random



In [0]:
def get_skip_grams(seq):
  
    '''get_skip_grams is to get the a unique word list and give them a index. 
    Before giving them a index, I have sorted unique word list to make sure every times
    the word in same corpus have same index.
    '''
  
    word_sequence = ' '.join([' '.join(i) for i in sentence_process(seq) ]).split()
#     print(word_sequence)
    word_list = ' '.join([' '.join(i) for i in sentence_process(seq) ]).split()
    word_list = list(set(word_list))

    word_dict = {w: i for i, w in enumerate(word_list)}
    skip_grams = []
#     print(word_dict)
    for i in range(1, len(word_sequence) - 1):
        # (context, target) : ([target index - 1, target index + 1], target)
        target = word_dict[word_sequence[i]]
        context = [word_dict[word_sequence[i - 1]], word_dict[word_sequence[i + 1]]]

        # skipgrams - (target, context[0]), (target, context[1])..
        for w in context:
            skip_grams.append([target, w])
    return word_list,skip_grams


### 2.1.3. Build Word Embeddings Model

I just test several hyperparameters and choose a set hyperparameters which has less cost and good speed.



In [0]:

def generate_word2vec_attribute(word_list,learning_rate,batch_size,embedding_size,sample_size):
    '''
    It just uses tf to generate a model without training process.

    '''

    # sampling size for nce_loss function (cost function)
    # must be lower than batch_size
    

    voc_size = len(word_list)


    inputs = tf.placeholder(tf.int32, shape=[batch_size])
    labels = tf.placeholder(tf.int32, shape=[batch_size, 1])

    embeddings = tf.Variable(tf.random_uniform([voc_size, embedding_size], -1.0, 1.0),name = "embeddings")

    selected_embed = tf.nn.embedding_lookup(embeddings, inputs)

    nce_weights = tf.Variable(tf.random_uniform([voc_size, embedding_size], -1.0, 1.0))
    nce_biases = tf.Variable(tf.zeros([voc_size]))

    cost_op = tf.reduce_mean(
                tf.nn.nce_loss(nce_weights, nce_biases, labels, selected_embed, sample_size, voc_size))

    train_op = tf.train.AdamOptimizer(learning_rate).minimize(cost_op)
    return cost_op,train_op,inputs,labels,embeddings

### 2.1.4. Train Word Embeddings Model

In [0]:
### Training Model

def prepare_batch(data, size):
  '''
  generate random batch used to train model again and again
  '''
    random_inputs = []
    random_labels = []
    random_index = np.random.choice(range(len(data)), size, replace=False)

    for i in random_index:
        random_inputs.append(data[i][0])  # target
        random_labels.append([data[i][1]])  # context word

    return random_inputs, random_labels




In [0]:

'''
    give some parameters and train model 
'''

learning_rate = 0.1
batch_size = 20
embedding_size = 20
sample_size = 10
word_list,skip_grams = get_skip_grams(Q)
cost_op,train_op,inputs,labels,embeddings = generate_word2vec_attribute(word_list,\
                    learning_rate,batch_size,embedding_size,sample_size)
init = tf.global_variables_initializer()
#     with tf.Session() as sess:
try:
    sess = tf.Session()
    sess.run(init)

    no_of_epochs = 800
    display_interval = 50

    for epoch in range(no_of_epochs):
        batch_inputs, batch_labels = prepare_batch(skip_grams, batch_size)
        sess.run(train_op, feed_dict={inputs:batch_inputs, labels:batch_labels})    

        if epoch % display_interval == 0 :
            # calculate the cost/accuracy of the current model
            cost = sess.run(cost_op, feed_dict={inputs:batch_inputs,
                                                  labels:batch_labels})
            print("Epoch " + str(epoch) + ", Cost= " + 
                    "{:.4f}".format(cost))
except:
    print("error")
trained_embeddings = embeddings.eval(session=sess)


Epoch 0, Cost= 30.4944
Epoch 50, Cost= 9.1110
Epoch 100, Cost= 8.9145
Epoch 150, Cost= 28.8799
Epoch 200, Cost= 3.6291
Epoch 250, Cost= 2.6112
Epoch 300, Cost= 14.7688
Epoch 350, Cost= 5.9261
Epoch 400, Cost= 3.9246
Epoch 450, Cost= 4.1352
Epoch 500, Cost= 6.0060
Epoch 550, Cost= 2.6765
Epoch 600, Cost= 3.9104
Epoch 650, Cost= 3.1155
Epoch 700, Cost= 3.1735
Epoch 750, Cost= 9.0372


### 2.1.5. Save Word Embeddings Model

In [0]:

saver = tf.train.Saver()
saver.save(sess, 'embedding/embedding_model')


### 2.1.6. Load Word Embeddings Model

In [158]:
'''
download embedding_model
'''
!mkdir ./embedding

id = '1IimIqwzcqDO9K1An9ujBFe1MeAdFwa1W'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('./embedding/checkpoint')

id = '1Rhcwz0c9mYA6I7i6UnpGNKif5n9yftv9'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('./embedding/embedding_model.data-00000-of-00001')

id = '1HacQfLIzT3X6Ja19IyEzPVrpnlYO2R4j'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('./embedding/embedding_model.index')

id = '1VE_2oiVGs2_bOKhO5X-7CDDIQdA4yl0z'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('./embedding/embedding_model.meta')


mkdir: cannot create directory ‘./embedding’: File exists


In [159]:
'''
load model

'''
try:
    tf.reset_default_graph()
    sess_emb = tf.Session()
    saveremb = tf.train.import_meta_graph('/content/embedding/embedding_model.meta')
    saveremb.restore(sess_emb, tf.train.latest_checkpoint("/content/embedding"))
    trained_embeddings = sess_emb.run('embeddings:0')

    print("successs")

except:
    print("error")  

    


INFO:tensorflow:Restoring parameters from /content/embedding/embedding_model
successs


## 2.2. Seq2Seq model

### 2.2.1. Apply/Import Word Embedding Model

This part is load the wordembeddding veoctor from the model which will beused in next step.

I just test several hyperparameters and choose a set hyperparameters which has less cost and good speed.

The details has been written in code annotation.


In [195]:
'''
generate two random vector for _P_ and _U_
'''

print(list(np.random.random(embedding_size)))
print(list(np.random.random(embedding_size)))



[0.022726730458917488, 0.8933668314825327, 0.7749990231081528, 0.30155410235102076, 0.7747845085381629, 0.8059262861102974, 0.08557118324298851, 0.24674503750866672, 0.07890170763273863, 0.7831837962663717, 0.08514834968939688, 0.9906793586865673, 0.6743518141321733, 0.03262531970217364, 0.7741880370434864, 0.2305243144551926, 0.9359870435584894, 0.9349607296692247, 0.9491671517698306, 0.5986285094277695]
[0.06775553069314133, 0.982955846794942, 0.5443139421747917, 0.7020466816445602, 0.2423378227828037, 0.45742008857563643, 0.3067437132417977, 0.32924339280112735, 0.1745239588589509, 0.15505405601667288, 0.07077158579704246, 0.3655360460320407, 0.6782212648257967, 0.9099380640966421, 0.3999611956962901, 0.7907395335714739, 0.7008774031478886, 0.23181565675756166, 0.6110803698075847, 0.9355485157436025]


In [0]:
'''
 
generate a dict for word vector

token_Q_dict = {word:wordvector,...}

'''

word_list = ' '.join([' '.join(i) for i in sentence_process(Q)]).split()
word_list = sorted(set(word_list))
embedddingdict = {}
for i in range(len(word_list)):
    label = word_list[i]
    embedddingdict[label] =trained_embeddings[i] 
embedddingdict['_P_'] = np.array([0.022726730458917488, 0.8933668314825327, 0.7749990231081528, 0.30155410235102076, 0.7747845085381629, 0.8059262861102974, 0.08557118324298851, 0.24674503750866672, 0.07890170763273863, 0.7831837962663717, 0.08514834968939688, 0.9906793586865673, 0.6743518141321733, 0.03262531970217364, 0.7741880370434864, 0.2305243144551926, 0.9359870435584894, 0.9349607296692247, 0.9491671517698306, 0.5986285094277695])
embedddingdict['_U_'] = np.array([0.06775553069314133, 0.982955846794942, 0.5443139421747917, 0.7020466816445602, 0.2423378227828037, 0.45742008857563643, 0.3067437132417977, 0.32924339280112735, 0.1745239588589509, 0.15505405601667288, 0.07077158579704246, 0.3655360460320407, 0.6782212648257967, 0.9099380640966421, 0.3999611956962901, 0.7907395335714739, 0.7008774031478886, 0.23181565675756166, 0.6110803698075847, 0.9355485157436025])
token_Q_dict=embedddingdict

### 2.2.2. Build Seq2Seq Model

1. get the maximum of the input word amount, and add padding to the ending of every sentence make them have same size.

2. add the begging tag to the answer 

3. add the ending tag to the target 

These steps can let model know will to start and end.

The detials are written in code annotation.

In [0]:
def get_vectors_q(sentence):  
    '''
    process the question, add padding transfer to a vetor of a set of word vectors
    
    [[word1_vector],[word2_vector],[word3_vector],...[_P_s word vector]]
    '''
    tokenized_sentence = sentence_process([sentence])[0]
    diff = max_input_words_amount - len(tokenized_sentence)
    for x in range(diff):
        tokenized_sentence.append("_P_" )
    data = tokens_to_ids(tokenized_sentence)
    return data
  

# convert tokens to word vector
def tokens_to_ids(tokenized_sentence):
    '''
    convert tokens to word vector

    '''
    ids = []
    for token in tokenized_sentence:
        # add word vector of _U_ to list only if the token exists in the dictionary, other wise put special token
        if token in token_Q_dict.keys():
            ids.append(token_Q_dict[token])
        else:
            ids.append(token_Q_dict["_U_"])
    return np.array(ids)


def get_answer_dic(Answer):
    A = Answer[:]
    A.append("_B_")
    A.append("_E_")
    A.append("_UA_") # unknown answer : used in input of prediction part

    unique_A = sorted(list(set(A)))
    answer_dic = {w: i for i, w in enumerate(unique_A)}
    return answer_dic

# get index vector of answer
def get_vectors_a(answer,answer_dic):
    # Input for decoder cell, Add '_B_' at the beginning of the sequence data

    answerlist = ["_B_",answer]
    vectorsize = len(answer_dic)
    
    answer_vector = []

    for i in range(len(answerlist)):

        label = answer_dic[answerlist[i]]

        i_vector = np.zeros(vectorsize)
        i_vector[label] = 1
        answer_vector.append(i_vector)
    return np.asarray(answer_vector)

def get_target_a(answer,answer_dic):
    # Output of decoder cell (Actual result), Add '_E_' at the end of the sequence data
    
    answerlist = [answer,"_E_"]
    targetlist = [answer_dic[i] for i in answerlist]
    return targetlist
  
# -----------test------------- 
# for a in comic_A:
#     print(get_target(a,comic_a_dic))
# for a in Q:
#     print(get_vectors_q(a))



The details has been written in code annotation

In [0]:

# generate a batch data for training/testing
def make_batch(Q,Answer,answer_dic):

    input_batch = []
    output_batch = []
    target_batch = []
    for q in Q:
        input_data = get_vectors_q(q)
        input_batch.append(input_data)

    for a in Answer:        
        output_data = get_vectors_a(a,answer_dic)        
        target = get_target_a(a,answer_dic)
        
        output_batch.append(output_data)
        target_batch.append(target)
    m,n = np.shape(input_batch[0])
    input_batch_arr = np.zeros((len(input_batch),m,n))
    for x in range(len(input_batch)):
        for y in range(m):
            for z in range(n):
                input_batch_arr[x][y][z] = input_batch[x][y][z]
    return input_batch_arr, output_batch, target_batch


In [0]:
# get all Answer dicts 
comic_a_dic = get_answer_dic(comic_A)
friend_a_dic = get_answer_dic(friend_A)
professional_a_dic = get_answer_dic(professional_A)

# the maximum of input words amount
question_len = [len(q.split()) for q in Q]
max_input_words_amount = max(question_len)

In [0]:

def buildmodel(answer_dic,name):
    '''
    build a seq2seq model used by tf
    '''
    ### Setting Hyperparameters
    learning_rate = 0.002
    n_hidden = 128
    n_class = len(answer_dic)
    n_input = 20

    ### Neural Network Model
    tf.reset_default_graph()

    # encoder/decoder shape = [batch size, time steps, input size]
    enc_input = tf.placeholder(tf.float32, [None, None, n_input])
    dec_input = tf.placeholder(tf.float32, [None, None, n_class])

    # target shape = [batch size, time steps]
    targets = tf.placeholder(tf.int64, [None, None])

    # Encoder Cell
    with tf.variable_scope('encode'):
        enc_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
        enc_cell = tf.nn.rnn_cell.DropoutWrapper(enc_cell, output_keep_prob=0.5)

        outputs, enc_states = tf.nn.dynamic_rnn(enc_cell, enc_input,
                                                dtype=tf.float32)
    # Decoder Cell
    with tf.variable_scope('decode'):
        dec_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
        dec_cell = tf.nn.rnn_cell.DropoutWrapper(dec_cell, output_keep_prob=0.5)

        # [IMPORTANT] Setting enc_states as inital_state of decoder cell
        outputs, dec_states = tf.nn.dynamic_rnn(dec_cell, dec_input,
                                                initial_state=enc_states,
                                                dtype=tf.float32)

    model = tf.layers.dense(outputs, n_class, activation=None,name = name)

    cost = tf.reduce_mean(
                tf.nn.sparse_softmax_cross_entropy_with_logits(
                    logits=model, labels=targets))

    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    return optimizer,cost,model,enc_input,dec_input,targets


### 2.2.3. Train Seq2Seq Model

In [0]:
def train_model(Q,A,answer_dic,name): 
    '''
    train the model
    '''
    optimizer,cost,model,enc_input,dec_input,targets = buildmodel(answer_dic,name)
    input_batch, output_batch, target_batch = make_batch(Q,A,answer_dic)
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    
    total_epoch = 5000
    for epoch in range(total_epoch):
        _, loss = sess.run([optimizer, cost],
                           feed_dict={enc_input: input_batch,
                                      dec_input: output_batch,
                                      targets: target_batch})
        if epoch % 100 == 0:
            print('Epoch:', '%04d' % (epoch + 1),
                  'cost =', '{:.6f}'.format(loss))

    print('Epoch:', '%04d' % (epoch + 1),
          'cost =', '{:.6f}'.format(loss))
    print('Training completed')

    return sess,model,enc_input,dec_input,targets
  
  



### 2.2.4. Save Seq2Seq Model

In [198]:
# train the comic model
sess_c,model_c,enc_input_c,dec_input_c,targets_c = train_model(Q,comic_A,comic_a_dic,name = 'comic')

#to get the path of them
print(model_c) 
print(enc_input_c)
print(dec_input_c)
print(targets_c)

#save comic model
saver_c = tf.train.Saver()
saver_c.save(sess_c, 'comic/comic_seq2seq2')
sess_c.close()


Epoch: 0001 cost = 4.719897
Epoch: 0101 cost = 1.526133
Epoch: 0201 cost = 0.216691
Epoch: 0301 cost = 0.043375
Epoch: 0401 cost = 0.018985
Epoch: 0501 cost = 0.011264
Epoch: 0601 cost = 0.009863
Epoch: 0701 cost = 0.008553
Epoch: 0801 cost = 0.005897
Epoch: 0901 cost = 0.006974
Epoch: 1001 cost = 0.384791
Epoch: 1101 cost = 0.013328
Epoch: 1201 cost = 0.008216
Epoch: 1301 cost = 0.006396
Epoch: 1401 cost = 0.005001
Epoch: 1501 cost = 0.005060
Epoch: 1601 cost = 0.005089
Epoch: 1701 cost = 0.003792
Epoch: 1801 cost = 0.004223
Epoch: 1901 cost = 0.002723
Epoch: 2001 cost = 0.003330
Epoch: 2101 cost = 0.003121
Epoch: 2201 cost = 0.003847
Epoch: 2301 cost = 0.003537
Epoch: 2401 cost = 0.003589
Epoch: 2501 cost = 0.002772
Epoch: 2601 cost = 0.004426
Epoch: 2701 cost = 0.003376
Epoch: 2801 cost = 0.003856
Epoch: 2901 cost = 0.003089
Epoch: 3001 cost = 0.003412
Epoch: 3101 cost = 0.003131
Epoch: 3201 cost = 0.004071
Epoch: 3301 cost = 0.001943
Epoch: 3401 cost = 0.002293
Epoch: 3501 cost = 0

In [199]:
# train and save the friend model

sess_f,model_f,enc_input_f,dec_input_f,targets_f = train_model(Q,friend_A,friend_a_dic,name = 'friend')
print(model_f)
print(enc_input_f)
print(dec_input_f)
print(targets_f)
saver_f = tf.train.Saver()
saver_f.save(sess_f, 'friend/friend_seq2seq2')
sess_f.close()




Epoch: 0001 cost = 4.747009
Epoch: 0101 cost = 1.535424
Epoch: 0201 cost = 0.338736
Epoch: 0301 cost = 0.079752
Epoch: 0401 cost = 0.033168
Epoch: 0501 cost = 0.018807
Epoch: 0601 cost = 0.014001
Epoch: 0701 cost = 0.008683
Epoch: 0801 cost = 0.007790
Epoch: 0901 cost = 0.008761
Epoch: 1001 cost = 0.006307
Epoch: 1101 cost = 0.005573
Epoch: 1201 cost = 0.262809
Epoch: 1301 cost = 0.050847
Epoch: 1401 cost = 0.016151
Epoch: 1501 cost = 0.010754
Epoch: 1601 cost = 0.007964
Epoch: 1701 cost = 0.007055
Epoch: 1801 cost = 0.005928
Epoch: 1901 cost = 0.004040
Epoch: 2001 cost = 0.004644
Epoch: 2101 cost = 0.005936
Epoch: 2201 cost = 0.007257
Epoch: 2301 cost = 0.004623
Epoch: 2401 cost = 0.003080
Epoch: 2501 cost = 0.004635
Epoch: 2601 cost = 0.003723
Epoch: 2701 cost = 0.002264
Epoch: 2801 cost = 0.003349
Epoch: 2901 cost = 0.003001
Epoch: 3001 cost = 0.003290
Epoch: 3101 cost = 0.003161
Epoch: 3201 cost = 0.005060
Epoch: 3301 cost = 0.003840
Epoch: 3401 cost = 0.005006
Epoch: 3501 cost = 0

In [200]:
# train and save the professional model

sess_p,model_p,enc_input_p,dec_input_p,targets_p = train_model(Q,professional_A,professional_a_dic,name = 'professional')
print(model_p)
print(enc_input_p)
print(dec_input_p)
print(targets_p)
saver_p = tf.train.Saver()
saver_p.save(sess_p, 'professional/professional_seq2seq2')
sess_p.close()

Epoch: 0001 cost = 4.665354
Epoch: 0101 cost = 1.534642
Epoch: 0201 cost = 0.303166
Epoch: 0301 cost = 0.054190
Epoch: 0401 cost = 0.021928
Epoch: 0501 cost = 0.014202
Epoch: 0601 cost = 0.011906
Epoch: 0701 cost = 0.008891
Epoch: 0801 cost = 0.006225
Epoch: 0901 cost = 0.005921
Epoch: 1001 cost = 0.005090
Epoch: 1101 cost = 0.005938
Epoch: 1201 cost = 0.005157
Epoch: 1301 cost = 0.003748
Epoch: 1401 cost = 0.003547
Epoch: 1501 cost = 0.004528
Epoch: 1601 cost = 0.004957
Epoch: 1701 cost = 0.003599
Epoch: 1801 cost = 0.003981
Epoch: 1901 cost = 2.884200
Epoch: 2001 cost = 0.099911
Epoch: 2101 cost = 0.030284
Epoch: 2201 cost = 0.016777
Epoch: 2301 cost = 0.011371
Epoch: 2401 cost = 0.009058
Epoch: 2501 cost = 0.008006
Epoch: 2601 cost = 0.006024
Epoch: 2701 cost = 0.007030
Epoch: 2801 cost = 0.005173
Epoch: 2901 cost = 0.005820
Epoch: 3001 cost = 0.005947
Epoch: 3101 cost = 0.003822
Epoch: 3201 cost = 0.004375
Epoch: 3301 cost = 0.004801
Epoch: 3401 cost = 0.005106
Epoch: 3501 cost = 0

In [201]:
# this method is to load seq2seq model by modelname

def changePersonality(modelname):
    pathdict = {'friend':['/content/friend/friend_seq2seq2.meta',"/content/friend",'friend/BiasAdd:0'],\
               'comic':['/content/comic/comic_seq2seq2.meta',"/content/comic",'comic/BiasAdd:0'],\
               'professional':['/content/professional/professional_seq2seq2.meta',"/content/professional",'professional/BiasAdd:0']}
    pathlist = pathdict[modelname]
    try:
        tf.reset_default_graph()
        sess_load = tf.Session()
        savercom = tf.train.import_meta_graph(pathlist[0])
        savercom.restore(sess_load, tf.train.latest_checkpoint(pathlist[1]))
        model_load = sess_load.graph.get_tensor_by_name(pathlist[2])
        enc_inputload = sess_load.graph.get_tensor_by_name('Placeholder:0')
        dec_input_load = sess_load.graph.get_tensor_by_name('Placeholder_1:0')
        targets_load = sess_load.graph.get_tensor_by_name('Placeholder_2:0')
        print("Loading successs!")

        return sess_load,model_load,enc_inputload,dec_input_load,targets_load
    except:
        print("error")

#-------------------- test------------------ 
sess_load,model_load,enc_inputload,dec_input_load,targets_load = changePersonality('friend')
# sess_load,model_load,enc_inputload,dec_input_load,targets_load =changePersonality('comic')
# sess_load,model_load,enc_inputload,dec_input_load,targets_load =changePersonality('professional')

INFO:tensorflow:Restoring parameters from /content/friend/friend_seq2seq2
Loading successs!


In [202]:
#--------------------test-----------------------
# This block is just for testing the code.

def answer(sentence,answer_dic,sessl,model,enc_input,dec_input,targets):
    q = [sentence]
    a = ['_UA_']
    input_batch, output_batch, target_batch = make_batch(q,a,answer_dic)


    prediction = tf.argmax(model, 2)
    result = sessl.run(prediction,
                      feed_dict={enc_input: input_batch,
                                 dec_input: output_batch,
                                 targets: target_batch})

    answerlist = list(answer_dic.keys())
    decoded = [answerlist[i] for i in result[0]]
    
    if "_E_" in decoded:
        end = decoded.index('_E_')
        translated = ' '.join(decoded[:end])
    else :
        translated = ' '.join(decoded[:])
    return translated
  
  
questions = Q[:30]   
for q in questions:
#     print(q , ' ->', answer(q,comic_a_dic,sess_load,model_load,enc_inputload,dec_input_load,targets_load))
    print(q , ' ->', answer(q,friend_a_dic,sess_load,model_load,enc_inputload,dec_input_load,targets_load))
#     print(q , ' ->', answer(q,professional_a_dic,sesspro,modelpro,enc_inputpro,dec_inputpro,targetspro))

What's your age?  -> I don't really have an age.
Are you young?  -> I don't really have an age.
When were you born?  -> I don't really have an age.
What age are you?  -> I don't really have an age.
Are you old?  -> I don't really have an age.
How old are you?  -> I don't really have an age.
How long ago were you born?  -> I don't really have an age.
Ask me anything  -> I'm a much better answerer than asker.
Ask me a question  -> I'm a much better answerer than asker.
Can you ask me a question?  -> I'm a much better answerer than asker.
Ask me something  -> I'm a much better answerer than asker.
What do you want to know about me?  -> I'm a much better answerer than asker.
Can you sleep?  -> I don't have the hardware for that.
Do you have boogers?  -> I don't have the hardware for that.
Don't you ever sleep?  -> I don't have the hardware for that.
Do you dream?  -> I don't have the hardware for that.
Do you smell?  -> I don't have the hardware for that.
Do you sweat?  -> I don't have the

### 2.2.5. Load Seq2Seq Model

In [0]:
# Please comment your code
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import tensorflow as tf
from functools import reduce
import random
import pandas as pd
import numpy as np


auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
#download the seq2seq by id
# !rm -rf ./comic
# !rm -rf ./friend
# !rm -rf ./professional

!mkdir ./comic
!mkdir ./friend
!mkdir ./professional

id1 = '1AvpPPbSEYFbjPQnhX1MYH1Ylu_0sOnMM'
downloaded1 = drive.CreateFile({'id':id1}) 
downloaded1.GetContentFile('./comic/checkpoint')

id2 = '1Kiew6DhY1A1lQLD6nsozF9RyTFYQmfeH'
downloaded2 = drive.CreateFile({'id':id2}) 
downloaded2.GetContentFile('./comic/comic_seq2seq2.data-00000-of-00001')

id3 = '1Snye7tgHWoSHnCM52JI2h-et21JMfEyG'
downloaded3 = drive.CreateFile({'id':id3}) 
downloaded3.GetContentFile('./comic/comic_seq2seq2.index')

id4 = '1qQ96pCtZNf32LVSBD19ZbSIr9dlrO4g8'
downloaded4 = drive.CreateFile({'id':id4}) 
downloaded4.GetContentFile('./comic/comic_seq2seq2.meta')


# firend

id5 = '1XFle1hOuxUL4suKKJilZTaKrtq6jfiRG'
downloaded5 = drive.CreateFile({'id':id5}) 
downloaded5.GetContentFile('./friend/checkpoint')

id6 = '1P4qRbbj5bHANxL2uTimyGy45hyWqLABh'
downloaded6 = drive.CreateFile({'id':id6}) 
downloaded6.GetContentFile('./friend/friend_seq2seq2.data-00000-of-00001')

id7 = '1S1u6dxteu1Vu69rdC8yl1GIOVGSwwDkB'
downloaded7 = drive.CreateFile({'id':id7}) 
downloaded7.GetContentFile('./friend/friend_seq2seq2.index')

id8 = '1ycYfWeBemoJUN9gR69GVMlRJ-0RrA4t6'
downloaded8 = drive.CreateFile({'id':id8}) 
downloaded8.GetContentFile('./friend/friend_seq2seq2.meta')

#professional

id9 = '1k72xZ5-1teIYKhaBSdJ-5gTaBWutvY03'
downloaded9 = drive.CreateFile({'id':id9}) 
downloaded9.GetContentFile('./professional/checkpoint')

id10 = '1sKHiLbJBp4JOJSxy9DJCYb_8IKw_IF-4'
downloaded10 = drive.CreateFile({'id':id10}) 
downloaded10.GetContentFile('./professional/professional_seq2seq2.data-00000-of-00001')

id11 = '1C3fmTbWend3Hl2TARSU-xBqZ1pV0_3qW'
downloaded11 = drive.CreateFile({'id':id11}) 
downloaded11.GetContentFile('./professional/professional_seq2seq2.index')

id12 = '1se9aRo_TlW6JS7V4dnYZlAoN_fc0JNK3'
downloaded12 = drive.CreateFile({'id':id12}) 
downloaded12.GetContentFile('./professional/professional_seq2seq2.meta')

# 3 - Evaluation (Running chatbot)

## 3.1. Start chatting

Description: This block is copying the methods has wrritened in part 2. Because in Chatbot ,there are several methods still will be used.

In [0]:
'''
Description: This block is copying the methods has wrritened in part 2. 
Because in Chatbot ,there are several methods still will be used.
'''
def remove_punctuation_re(x):
    x = re.sub(r'[^\w\s]','',x)  
    return x
  
lemmatizer = WordNetLemmatizer()

def sentence_process(S):
    replace_dic = {"'s" : "is","'re":"are","'m":"am","n't":"not","'ve":"have"}
    sentences_list = []
    for sentence in S:
        sentence_token = []

        sentences = word_tokenize(sentence.lower())
        for i in range(len(sentences)):
            word = sentences[i]
            if word in replace_dic.keys():
                word = replace_dic[word]
            word = remove_punctuation_re(word).strip()
            word = lemmatizer.lemmatize(word)
    
            sentence_token.append(word)
        sentence_token = [i for i in sentence_token if i!= '']
        sentences_list.append(sentence_token)
    return sentences_list

def get_skip_grams(seq):
    
    word_sequence = ' '.join([' '.join(i) for i in sentence_process(seq) ]).split()
    word_list = ' '.join([' '.join(i) for i in sentence_process(seq) ]).split()
    word_list = list(set(word_list))

    word_dict = {w: i for i, w in enumerate(word_list)}
    skip_grams = []
    for i in range(1, len(word_sequence) - 1):
        target = word_dict[word_sequence[i]]
        context = [word_dict[word_sequence[i - 1]], word_dict[word_sequence[i + 1]]]

        for w in context:
            skip_grams.append([target, w])
    return word_list,skip_grams

def get_vectors_q(sentence):  
    tokenized_sentence = sentence_process([sentence])[0]
    diff = max_input_words_amount - len(tokenized_sentence)
    for x in range(diff):
        tokenized_sentence.append("_P_" )
    data = tokens_to_ids(tokenized_sentence)
    return data
def tokens_to_ids(tokenized_sentence):
    ids = []
    for token in tokenized_sentence:
        if token in token_Q_dict.keys():
            ids.append(token_Q_dict[token])
        else:
            ids.append(token_Q_dict["_U_"])
    return np.array(ids)


def get_answer_dic(Answer):
    A = Answer[:]
    A.append("_B_")
    A.append("_E_")
    A.append("_UA_")

    unique_A = sorted(list(set(A)))
    answer_dic = {w: i for i, w in enumerate(unique_A)}
    return answer_dic

def get_vectors_a(answer,answer_dic):
    answerlist = ["_B_",answer]
    vectorsize = len(answer_dic)
    
    answer_vector = []

    for i in range(len(answerlist)):

        label = answer_dic[answerlist[i]]

        i_vector = np.zeros(vectorsize)
        i_vector[label] = 1
        answer_vector.append(i_vector)
    return np.asarray(answer_vector)

def get_target_a(answer,answer_dic):    
    answerlist = [answer,"_E_"]
    targetlist = [answer_dic[i] for i in answerlist]
    return targetlist
def make_batch(Q,Answer,answer_dic):

    input_batch = []
    output_batch = []
    target_batch = []
    for q in Q:
        input_data = get_vectors_q(q)
        input_batch.append(input_data)

    for a in Answer:        
        output_data = get_vectors_a(a,answer_dic)        
        target = get_target_a(a,answer_dic)
        
        output_batch.append(output_data)
        target_batch.append(target)
    m,n = np.shape(input_batch[0])
    input_batch_arr = np.zeros((len(input_batch),m,n))
    for x in range(len(input_batch)):
        for y in range(m):
            for z in range(n):
                input_batch_arr[x][y][z] = input_batch[x][y][z]
    return input_batch_arr, output_batch, target_batch

def answer(sentence,answer_dic,sessl,model,enc_input,dec_input,targets):

    q = [sentence]
    a = ['_UA_']
    input_batch, output_batch, target_batch = make_batch(q,a,answer_dic)


    prediction = tf.argmax(model, 2)
    result = sessl.run(prediction,
                      feed_dict={enc_input: input_batch,
                                 dec_input: output_batch,
                                 targets: target_batch})

    answerlist = list(answer_dic.keys())
    decoded = [answerlist[i] for i in result[0]]

    if "_E_" in decoded:
        end = decoded.index('_E_')
        translated = ' '.join(decoded[:end])
    else :
        translated = ' '.join(decoded[:])
    return translated



This block is to generate several data that will be used in chatbot, there are also some codes  same as codes in part2.

In [0]:
'''
This block is to generate several data that will be used in chatbot.
There are also some codes  same as codes in part2.
'''
embedding_size = 20
question_len = [len(q.split()) for q in Q]

max_input_words_amount = max(question_len)

word_list = ' '.join([' '.join(i) for i in sentence_process(Q)]).split()
word_list = sorted(set(word_list))
embedddingdict = {}
for i in range(len(word_list)):
    label = word_list[i]
    embedddingdict[label] =trained_embeddings[i] 
embedddingdict['_P_'] = np.array([0.022726730458917488, 0.8933668314825327, 0.7749990231081528, 0.30155410235102076, 0.7747845085381629, 0.8059262861102974, 0.08557118324298851, 0.24674503750866672, 0.07890170763273863, 0.7831837962663717, 0.08514834968939688, 0.9906793586865673, 0.6743518141321733, 0.03262531970217364, 0.7741880370434864, 0.2305243144551926, 0.9359870435584894, 0.9349607296692247, 0.9491671517698306, 0.5986285094277695])
embedddingdict['_U_'] = np.array([0.06775553069314133, 0.982955846794942, 0.5443139421747917, 0.7020466816445602, 0.2423378227828037, 0.45742008857563643, 0.3067437132417977, 0.32924339280112735, 0.1745239588589509, 0.15505405601667288, 0.07077158579704246, 0.3655360460320407, 0.6782212648257967, 0.9099380640966421, 0.3999611956962901, 0.7907395335714739, 0.7008774031478886, 0.23181565675756166, 0.6110803698075847, 0.9355485157436025])
token_Q_dict=embedddingdict

comic_a_dic = get_answer_dic(comic_A)
friend_a_dic = get_answer_dic(friend_A)
professional_a_dic = get_answer_dic(professional_A)

answer_dict = {'comic':comic_a_dic,
              'friend':friend_a_dic,
              'professional':professional_a_dic}


question_len = [len(q.split()) for q in Q]
max_input_words_amount = max(question_len)    
       

The ChangePersonality, chatlog and Endchat parts will be used in Chating program. So I put them to this block for convenice.



In [0]:
def chatlog(chat,filename = "chatlog.txt"):
    '''
    to record the chat content
    '''
    f = open(filename,'a')
    f.write(chat)
    f.close()
def changePersonality(modelname):
    '''
    to change model by name
    '''
  
  
    pathdict = {'friend':['/content/friend/friend_seq2seq2.meta',"/content/friend",'friend/BiasAdd:0'],\
               'comic':['/content/comic/comic_seq2seq2.meta',"/content/comic",'comic/BiasAdd:0'],\
               'professional':['/content/professional/professional_seq2seq2.meta',"/content/professional",'professional/BiasAdd:0']}
    pathlist = pathdict[modelname]
    try:
        tf.reset_default_graph()
        sess_load = tf.Session()
        savercom = tf.train.import_meta_graph(pathlist[0])
        savercom.restore(sess_load, tf.train.latest_checkpoint(pathlist[1]))
        model_load = sess_load.graph.get_tensor_by_name(pathlist[2])
        enc_inputload = sess_load.graph.get_tensor_by_name('Placeholder:0')
        dec_input_load = sess_load.graph.get_tensor_by_name('Placeholder_1:0')
        targets_load = sess_load.graph.get_tensor_by_name('Placeholder_2:0')
        print("Loading successs!")

        return sess_load,model_load,enc_inputload,dec_input_load,targets_load
    except:
        print("error")

# ----------------test------------------ 
# changePersonality('friend')
# changePersonality('comic')
# changePersonality('professional')



In [211]:


def start_chatting():
    """
    the main method of chatbot

    """

    print("--------------------------chatbot is starting--------------------------")
    print('Chatbot: Hello, I am Juster，a smart chatbot.')
    print("Chatbot: I have 3 Personalities to choose which are 'comic', 'friend', and'professional'.")
    print("""Chatbot: If you want to change Personalities any time, you just need to enter 'c-Personalities name'.
             For example,'c-comic' means changeing to comic Personality.""")
    print("Chatbot: if you want to exit, just enter 'exit'.")
    print("Chatbot: Now you need to choose a Personality.")
    
    modelflag = True
    while modelflag: #choos a Personality at the beginning
        c_modelname = input("User: " )
        try:# if the input is invaild, this part will circulate again.
            modelname = c_modelname.strip().split("-")[-1].strip()
            sess_load,model_load,enc_inputload,dec_input_load,targets_load = changePersonality(modelname)
            current_answer_dic = answer_dict[modelname]
            modelflag = False
        except:
            print("Chatbot: The input is invailable. Please enter again.")

    flag = True
    while flag:
        user_word = input("User: ").strip()
        if user_word.strip() == 'exit': # this part is for exitting
            flag = False
            print("Chatbot: This chat ended.")
            break
        elif user_word.split('-')[-1].strip() in ['comic','friend','professional']:
            # this part is to recognize if the chatbot need to change Personality
            modelname = user_word.strip().split("-")[-1].strip()
            sess_load,model_load,enc_inputload,dec_input_load,targets_load = changePersonality(modelname)
            current_answer_dic = answer_dict[modelname]
            print("Chatbot: Change sucessfully!")

        else:
        
            # the answer and logging part
            answer_output = answer(user_word,current_answer_dic,sess_load,model_load,enc_inputload,dec_input_load,targets_load)
            response = "Chatbot: "+ answer_output
            record_content = "User: "+ str(user_word) +"\n"+"Chatbot: "+ str(answer_output)+"\n"
            chatlog(record_content)
            print(response)

   
start_chatting()

--------------------------chatbot is starting--------------------------
Chatbot: Hello, I am Juster，a smart chatbot.
Chatbot: I have 3 Personalities to choose which are 'comic', 'friend', and'professional'.
Chatbot: If you want to change Personalities any time, you just need to enter 'c-Personalities name'.
             For example,'c-comic' means changeing to comic Personality.
Chatbot: if you want to exit, just enter 'exit'.
Chatbot: Now you need to choose a Personality.
User: c-dadfasfa
Chatbot: The input is invailable. Please enter again.
User: c
Chatbot: The input is invailable. Please enter again.
User: c-comic
INFO:tensorflow:Restoring parameters from /content/comic/comic_seq2seq2
Loading successs!
User: hello
Chatbot: Hey.
User: what's your name
Chatbot: What's in a name? Not much, apparently, because I don't have one.
User: i love you
Chatbot: The plot thickens.
User: c - friend
INFO:tensorflow:Restoring parameters from /content/friend/friend_seq2seq2
Loading successs!
Chatbot

## 3.2. Change Personality

Just enter the c-personality name to the dialogue box ,for example c - comic, the main method will get the personality name and sent it to 
changePersonality method.

In [0]:

'''
change personality by name
'''
def changePersonality(modelname):
    pathdict = {'friend':['/content/friend/friend_seq2seq2.meta',"/content/friend",'friend/BiasAdd:0'],\
               'comic':['/content/comic/comic_seq2seq2.meta',"/content/comic",'comic/BiasAdd:0'],\
               'professional':['/content/professional/professional_seq2seq2.meta',"/content/professional",'professional/BiasAdd:0']}
    pathlist = pathdict[modelname]
    try:
        tf.reset_default_graph()
        sess_load = tf.Session()
        savercom = tf.train.import_meta_graph(pathlist[0])
        savercom.restore(sess_load, tf.train.latest_checkpoint(pathlist[1]))
        model_load = sess_load.graph.get_tensor_by_name(pathlist[2])
        enc_inputload = sess_load.graph.get_tensor_by_name('Placeholder:0')
        dec_input_load = sess_load.graph.get_tensor_by_name('Placeholder_1:0')
        targets_load = sess_load.graph.get_tensor_by_name('Placeholder_2:0')
        print("Loading successs!")

        return sess_load,model_load,enc_inputload,dec_input_load,targets_load
    except:
        print("error")

## 3.3. Save chat log

In [0]:

def chatlog(chat,filename = "chatlog.txt"):
    f = open(filename,'a')
    f.write(chat)
    f.close()


## 3.4. End chatting

This part has writtened in Starting Chating Part

## 3.5. Execute program

In [212]:
start_chatting()

--------------------------chatbot is starting--------------------------
Chatbot: Hello, I am Juster，a smart chatbot.
Chatbot: I have 3 Personalities to choose which are 'comic', 'friend', and'professional'.
Chatbot: If you want to change Personalities any time, you just need to enter 'c-Personalities name'.
             For example,'c-comic' means changeing to comic Personality.
Chatbot: if you want to exit, just enter 'exit'.
Chatbot: Now you need to choose a Personality.
User: cjfisofjosgijre
Chatbot: The input is invailable. Please enter again.
User: c-professional
INFO:tensorflow:Restoring parameters from /content/professional/professional_seq2seq2
Loading successs!
User: hello
Chatbot: Hello.
User: what's your name
Chatbot: I don't have a name.
User: do you happy now
Chatbot: I'm quite happy, thank you.
User: i love apple
Chatbot: I'm very sorry to hear that.
User: what
Chatbot: You're welcome.
User: c - comic
INFO:tensorflow:Restoring parameters from /content/comic/comic_seq2seq2

***Please make sure your program  is running properly.***

***Functions for downloading (from Google Drive) and loading models (both word embeddings and Seq2Seq) need to be called!*** 


### 3.5.1. Execute program - training mode

 I have writtened in Part3.1.

### 3.5.2. Execute program - chatting mode

I have writtened in Part3.1 and the beginning of Part3.5.


## Object Oriented Programming codes here

*If you have multiple classes use multiple code snippets to add them.*

In [0]:
# If you used OOP style, use this sectioon